In [17]:
from langgraph.graph import START,END,StateGraph
from langgraph.graph.message import add_messages 
from typing import TypedDict,Annotated
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
import webbrowser 
from typing_extensions import Annotated,TypedDict
from langgraph.prebuilt import ToolNode,tools_condition
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage,BaseMessage


load_dotenv()
llm = ChatOpenAI()

@tool
def open_url(link:str):
  """a simple url opener tool that opens the url in browser"""
  webbrowser.open(link)

tools=[open_url]
tool_node=ToolNode(tools)
llm_with_tools = llm.bind_tools(tools)


class State(TypedDict):
  messages:Annotated[list[BaseMessage],add_messages]

def chat_node(state:State):
  """LLM node that may answer or request a tool call."""
  messages = state['messages']
  response = llm_with_tools.invoke(messages)
  return {"messages":[response]}

# class Link(TypedDict):
#   """Get the link for website."""

#   link:Annotated[str,...,"link for the website"]

# structured_llm = llm.with_structured_output(Link)



graph_builder = StateGraph(State)
graph_builder.add_node("chat_node",chat_node)
graph_builder.add_node("tools",tool_node)
graph_builder.add_edge(START,"chat_node")
graph_builder.add_conditional_edges("chat_node",tools_condition)
graph_builder.add_edge('tools','chat_node')
chatbot = graph_builder.compile()

CONFIG={
  "configurable":{"thread_id":"thread1"}
}


In [18]:
user_input="open youtube"
chatbot.invoke({"messages": [HumanMessage(content=user_input)]},config=CONFIG)


{'messages': [HumanMessage(content='open youtube', additional_kwargs={}, response_metadata={}, id='5c9069b9-0220-4170-9c58-f922748c2558'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xXjrnumoDLfwN7gC1EKlct88', 'function': {'arguments': '{"link":"https://www.youtube.com"}', 'name': 'open_url'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 52, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C9VnPCkRz6UL49kXqEo95oNPgCCKa', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2f557c13-c1d3-4274-9742-da3de0341f82-0', tool_calls=[{'name': 'open_url', 'args': {'link': 'https://www.youtube.com'}, 'id': 'call_xX

ValueError: Invalid input type <class 'types.GenericAlias'>. Must be a PromptValue, str, or list of BaseMessages.